In [1]:
!pip install psycopg2-binary

In [2]:
import numpy as np
import pandas as pd
import os, glob
import sqlalchemy
import sqlite3
from sqlite3 import Error
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
import gzip
import csv
import matplotlib.pyplot as plt
import psycopg2

# Connecting to mimic postgres database

In [ ]:
from sqlalchemy import create_engine

# Postgres username, password, database name
# @team - you'll need to change this according to what your Postgres info is under the Properties tab of your server in PgAdmin
POSTGRES_ADDRESS = '127.0.0.1'
POSTGRES_PORT= '5439'
POSTGRES_USERNAME= 'postgres'
POSTGRES_PASSWORD= 'password'
POSTGRES_DBNAME = 'mimic'

# putting in info
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)


# Feature extraction and cleaning by group

In [ ]:
# Albumin

In [ ]:
# Anion Gap (calculated from Na, K, Cl, HCO3)

In [ ]:
# Bilirubin

In [ ]:
# Chloride

In [ ]:
# Creatinine

In [ ]:
# Diastolic BP

In [ ]:
# Epinephrine

In [ ]:
# Glasgow Coma Scale

In [ ]:
# Glucose

In [ ]:
# Heart rate

In [ ]:
# Hemoglobin

In [ ]:
# Intubation

In [ ]:
# Lactate

In [ ]:
# Mean BP

In [ ]:
# Mechanical ventilation

In [ ]:
# pH

In [ ]:
# Platelet

In [ ]:
# Potassium

In [ ]:
# Respiration rate

In [ ]:
# Sodium

In [ ]:
# SOFA Score ?? ? ?  - Still need to get feedback from mentors re: inclusion

In [ ]:
# SpO2

In [ ]:
# Systolic BP

In [ ]:
# Temperature

In [ ]:
# Urine output

In [ ]:
# White blood count